# Assignment 1
## Question `1` (K-Nearest Neighbour)

| | |
|-|-|
| Course | Statistical Methods in AI |
| Release Date | `19.01.2023` |
| Due Date | `29.01.2023` |

### Instructions:
1.   Assignment must be implemented using python notebook only (Colab , VsCode , Jupyter etc.)
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for algorithms as well (sklearn etc). You are not however allowed to directly use classifier models.
3.   The performance of the model will hold weightage but you will also be graded largely for data preprocessing steps , explanations , feature selection for vectors etc.
4.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

### The Dataset
The dataset is to be downloaded from the following drive link ([Link](https://drive.google.com/file/d/1u55iIrTrn41n2lv8HBjtdKLhDcy_6s7O/view?usp=sharing)).
The dataset is a collection of *11099 tweets and has 31 columns*. The data will be in the form of a csv file which you can load in any format. The ground truth is available in the following drive link ([Link](https://drive.google.com/file/d/1--cozM5hXDwdbbDaWlB-8NqwSj0nh1Kg/view?usp=sharing)) which corresponds to whether a tweet was popular or not. Since the task involves selecting features yourself to vectorize a tweet , we suggest some data analysis of the columns you consider important.
<br><br>

### The Task
You have to build a classifier which can predict the popularity of the tweet, i.e , if the tweet was popular or not. You are required to use **KNN** algorithm to build the classifier and cannot use any inbuilt classifier. All columns are supposed to be analyzed , filtered and preprocessed to determine its importance as a feature in the vector for every tweet (Not every column will be useful).<br>
The Data contains the **raw text of the tweet**(in the text column) as well as other **meta data** like likes count , user followers count. Note that it might be useful to **create new columns** with useful information. For example, *number of hashtags* might be useful but is not directly present as a column.<br>
There are 3 main sub parts:
1. *Vectorize tweets using only meta data* - likes , user followers count , and other created data
2. *Vectorize tweets using only it's text*. This segment will require NLP techniques to clean the text and extract a vector using a BoW model. Here is a useful link for the same - [Tf-Idf](https://towardsdatascience.com/text-vectorization-term-frequency-inverse-document-frequency-tfidf-5a3f9604da6d). Since these vectors will be very large , we recommend reducing their dimensionality (~10 - 25). Hint: [Dimentionality Reduction](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491). Please note that for this also you are allowed to use libraries.

3. *Combining the vectors from above two techinques to create one bigger vector*
<br>


Using KNN on these vectors build a classifier to predict the popularity of the tweet and report accuracies on each of the three methods as well as analysis. You can use sklearn's Nearest Neighbors and need not write KNN from scratch. (However you cannot use the classifier directly). You are expected to try the classifier for different number of neighbors and identify the optimal K value.

## Import necessary libraries

In [112]:
import pandas as pd
import numpy as np
import json
import matplotlib
import math
from tqdm import tqdm
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Load and display the data

In [113]:
data = pd.read_json("tweets.json", lines=True)
gt = np.loadtxt("ground_truth.csv")
print(gt)

print("The columns are as follows:")
for i in data.columns:
    print(i, end=", ")

data.insert(0, "itemID", range(0, len(data)))


pd.options.display.max_seq_items = 4000

print("\n\nSample data:")

a = 0
for i,j in zip(data.keys(), data.values[1]):
    print(i, " : ", j)

print("\n\n")
for x in range(len(data.values)):
    if gt[x] == 1:
        for i,j in zip(data.keys(), data.values[x]):
            print(i, " : ", j)
        break


[0. 0. 1. ... 0. 0. 0.]
The columns are as follows:
created_at, id, id_str, text, truncated, entities, metadata, source, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str, in_reply_to_screen_name, user, geo, coordinates, place, contributors, retweeted_status, is_quote_status, retweet_count, favorite_count, favorited, retweeted, lang, possibly_sensitive, quoted_status_id, quoted_status_id_str, extended_entities, quoted_status, withheld_in_countries, 

Sample data:
itemID  :  1
created_at  :  2018-07-31 13:34:40+00:00
id  :  1024287229512953856
id_str  :  1024287229512953856
text  :  @hail_ee23 Thanks love its just the feeling of eyes that get me so nervous ❤️
truncated  :  False
entities  :  {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'hail_ee23', 'name': 'Jordan Vaughn', 'id': 927185727053553665, 'id_str': '927185727053553665', 'indices': [0, 10]}], 'urls': []}
metadata  :  {'iso_language_code': 'en', 'result_type': 're

## Exploratory Data Analysis
*This is an ungraded section but is recommended to get a good grasp on the dataset*

In [114]:
# your code here

## Part-1
*Vectorize tweets using only meta data*

In [115]:
# relevant metadata columns
# relCols = ["id", "fcount", "frndcount", "lcount", "user favcount", "verified", "scount", "rt count", "fav count", "sens"]
relCols = ["id", "fcount",  "rt count", "fav"] #,  "frncount", "fav count", ]

# return data and test splits
def getFeatures(data, percent=0.9):
  """
  Funtion to return a matrix of dimensions (number of tweets, number of chosen features)
  Input parameters to this funcion are to be chosen as per requirement (Example: Loaded dataframe of the dataset) 
  """

  """
  relevant columns:
  follower count
  friend count
  listed_count
  favourite count
  verified
  statuses_count
  rt count
  fac count
  sensitive
  """

  vecWidth = len(relCols)
  vecHeight = int(len(data.values))   # for data

  dataFeatureVector = np.zeros((vecHeight, vecWidth), dtype=np.int32)

  columns = list(data.columns)


  testCount = 0
  for i, item in enumerate(data.values):    
    feature = np.zeros(vecWidth)

    feature[0] = item[columns.index("itemID")]                                                    # ID
    feature[1] = item[columns.index("retweet_count")]
    feature[2] = item[columns.index("user")]["followers_count"]
    # feature[3] = item[columns.index("user")]["friends_count"]
    feature[3] = item[columns.index("favorite_count")]
    # feature[5] = item[columns.index("user")]["friends_count"]
    # feature[4] = item[columns.index("user")]["verified"]
    # feature[7] = item[columns.index("user")]["statuses_count"]
    # feature[8] = item[columns.index("retweet_count")]
    # feature[9] = item[columns.index("favorite_count")]
    
    dataFeatureVector[i] = feature

  return dataFeatureVector

data = data.sample(frac=1)

fullVector = getFeatures(data)
featureVector = fullVector[:int(0.9*fullVector.shape[0])]
testFeatureVector = fullVector[int(0.9*fullVector.shape[0]):]

print("Metadata only feature vector for the data split: \n", featureVector, featureVector.shape)
print("Metadata only feature vector for the test split: \n", testFeatureVector, testFeatureVector.shape)


Metadata only feature vector for the data split: 
 [[  430     0   115     0]
 [  719   108   720     0]
 [ 1103   169    67     0]
 ...
 [ 5977 28127   503     0]
 [ 7310     0     5     0]
 [ 4108     0   387     1]] (9989, 4)
Metadata only feature vector for the test split: 
 [[10715   196   207     0]
 [ 4376  2905  1724     0]
 [10009    28   114     0]
 ...
 [ 1618     0   561     0]
 [  613 17373   345     0]
 [ 6950   530   320     0]] (1110, 4)


Perform KNN using the vector obtained from get_features() function. Following are the steps to be followed:
1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values. 

In [116]:
# Part 1 (normalising)

def normaliseFeatureVector(featureVector):
    normFeatureVector = np.zeros_like(featureVector, dtype=np.float32)

    _, nCols = featureVector.shape

    normFeatureVector[:, 0] = featureVector[:, 0] # ID
    for i in range(1, nCols):
        sum = np.sum(featureVector[:, i])

        if sum != 0:
            normFeatureVector[:, i] = (featureVector[:, i] - np.mean(featureVector[:, i])) / (sum/2.0)

        else:
            print("Zero sum for column %i" % i)
            normFeatureVector[:, i] = (featureVector[:, i] - np.mean(featureVector[:, i]))

    return normFeatureVector

# Part 2 (split)
# using a 90/10 training(data)/test(eval) split

fullVector = getFeatures(data, percent=0.7)
dataFeatureVector = fullVector[:int(0.9*fullVector.shape[0])]
testFeatureVector = fullVector[int(0.9*fullVector.shape[0]):]

nDataFeatureVector = normaliseFeatureVector(dataFeatureVector)
nTestFeatureVector = normaliseFeatureVector(testFeatureVector)

print("Normalised data feature vector:\n", nDataFeatureVector, "\n")
print("Normalised test feature vector:\n", nTestFeatureVector, "\n")


Normalised data feature vector:
 [[ 4.3000000e+02 -2.0022024e-04 -1.9658102e-04 -2.0022024e-04]
 [ 7.1900000e+02 -1.9250841e-04 -1.7743558e-04 -2.0022024e-04]
 [ 1.1030000e+03 -1.8815264e-04 -1.9810001e-04 -2.0022024e-04]
 ...
 [ 5.9770000e+03  1.8082140e-03 -1.8430263e-04 -2.0022024e-04]
 [ 7.3100000e+03 -2.0022024e-04 -2.0006202e-04 -2.0022024e-04]
 [ 4.1080000e+03 -2.0022024e-04 -1.8797349e-04  4.7258879e-03]] 

Normalised test feature vector:
 [[ 1.0715000e+04 -1.6629746e-03 -1.6833582e-03 -1.8018018e-03]
 [ 4.3760000e+03  2.5581502e-04 -8.1534416e-04 -1.8018018e-03]
 [ 1.0009000e+04 -1.7819693e-03 -1.7365720e-03 -1.8018018e-03]
 ...
 [ 1.6180000e+03 -1.8018018e-03 -1.4808025e-03 -1.8018018e-03]
 [ 6.1300000e+02  1.0503526e-02 -1.6043958e-03 -1.8018018e-03]
 [ 6.9500000e+03 -1.4264018e-03 -1.6187006e-03 -1.8018018e-03]] 



In [117]:
# Part 3 (fitting and determining k)

def calcL2Norm(a, b):
    if len(a) != len(b):
        print("Incorrect dim")
        exit(1)
    
    sum = 0
    for x in range(1,len(a)):
        sum += (a[x]-b[x]) * (a[x]-b[x])
    
    return math.sqrt(sum)


def kNN(k, nDataFeatureVector, nTestFeatureVector):
    correctPredictions = 0
    correctPositive = 0
    totalPredictions = len(nTestFeatureVector)
    totalPositives = 0
    
    pptp = 0
    pntp = 0
    pptn = 0
    pntn = 0

    # get the features with the k smallest costs
    for testEntry in tqdm(nTestFeatureVector):
        allCosts = nDataFeatureVector - testEntry
        allCosts = np.square(allCosts)
        allCosts = np.sum(allCosts[:, 1:], axis=1)        

        sortedCosts = np.zeros((allCosts.shape[0], 2))
        sortedCosts[:,0] = np.sqrt(allCosts)
        sortedCosts[:,1] = np.arange(len(nDataFeatureVector))                

        ind = np.argsort(sortedCosts[:,0])
        sortedCosts = sortedCosts[ind]

        closest = sortedCosts[:k]
        # check nbd classes
        tot = 0.0
        predictedClass = 0

        bleep = 0
        for i in closest:
            # print(i.shape)
            # print(i)
            # print(gt[int(i[1])])
            if gt[int(i[1])] == 1.0:
                bleep += 1
            tot += gt[int(i[1])]
        
        # if bleep > 0:
            # print(bleep, testEntry[0])

        if float(tot)/k < 0.5:
            predictedClass = 0
        else:
            predictedClass = 1

        if gt[int(testEntry[0])] == predictedClass:
            correctPredictions += 1
            if predictedClass == 1:
                correctPositive += 1

        if gt[int(testEntry[0])] == 1:
            totalPositives += 1

        if predictedClass == 1:
            if gt[int(testEntry[0])] == 0:
                pptn += 1
            else:
                pptp += 1

        else:
            if gt[int(testEntry[0])] == 0:
                pntn += 1
            else:
                pntp += 1
    
    print("pptp: ", pptp, " | pptn: ", pptn, " | pntp: ", pntp, " | pntn: ", pntn)
    accuracy = float(pptp + pntn)/(pptp+pptn+pntp+pntn)
    positiveAccuracy = float(pptp)/(pntp + pptp)
    return accuracy, positiveAccuracy, pptp, (pptp+pntp)

# loop over all ks, calculate accuracy and percent of correctly predicted positives
maxK = 10
for k in range(1, maxK):
    accuracy, positiveAccuracy, correctPositive, tp = kNN(k, nDataFeatureVector, nTestFeatureVector)
    print("K = ", k, " | Accuracy = ", accuracy, " | Correctly predicted positives = ", positiveAccuracy, " | kp = ", correctPositive, " out of ", tp)
        


100%|██████████| 1110/1110 [00:00<00:00, 1288.62it/s]


pptp:  8  | pptn:  77  | pntp:  96  | pntn:  929
K =  1  | Accuracy =  0.8441441441441442  | Correctly predicted positives =  0.07692307692307693  | kp =  8  out of  104


100%|██████████| 1110/1110 [00:00<00:00, 1266.35it/s]


pptp:  18  | pptn:  172  | pntp:  86  | pntn:  834
K =  2  | Accuracy =  0.7675675675675676  | Correctly predicted positives =  0.17307692307692307  | kp =  18  out of  104


100%|██████████| 1110/1110 [00:00<00:00, 1283.32it/s]


...

pptp:  0  | pptn:  0  | pntp:  104  | pntn:  1006
K =  7  | Accuracy =  0.9063063063063063  | Correctly predicted positives =  0.0  | kp =  0  out of  104


100%|██████████| 1110/1110 [00:00<00:00, 1291.38it/s]


pptp:  0  | pptn:  0  | pntp:  104  | pntn:  1006
K =  8  | Accuracy =  0.9063063063063063  | Correctly predicted positives =  0.0  | kp =  0  out of  104


100%|██████████| 1110/1110 [00:00<00:00, 1286.60it/s]

pptp:  0  | pptn:  0  | pntp:  104  | pntn:  1006
K =  9  | Accuracy =  0.9063063063063063  | Correctly predicted positives =  0.0  | kp =  0  out of  104


For the chosen metadata fields, the optimal value of k seems to be 2.
The actual accuracy is not a good metric due to how heavily skewed the data is to not popular tweets. Instead, the number of correctly predicted positives is included as a metric.
Past a certain value of k, the number of non popular tweets in the nearest neighbours greatly outweighs the closeby positive tweets, thus no tweet is ever predicted to be popular.

## Part-2
Vectorize tweets based on the text. More details and reference links can be checked on the Tasks list in the start of the notebook

In [118]:

def tweet_vectoriser(data, percent=0.9):
  """
  Funtion to return a matrix of dimensions (number of tweets, number of features extracted per tweet)
  Following are the steps for be followed:
    1. Remove links, tags and hashtags from each tweet.
    2. Apply TF-IDF on the tweets to extract a vector. 
    3. Perform dimensionality reduction on the obtained vector. 
  Input parameters to this funcion are to be chosen as per requirement (Example: Array of tweets) 
  """
  
  # clean text
  cleanText = []
  columns = list(data.columns)

  for i in data.values:
    text = i[columns.index("text")]

    removalRegex = "@[a-z0-9]*|[?!\.\*]*| [a-z0-9]*…$|^rt|[ ]{2,}|https:\/\/.*|\&amp|\\n"
    leaveNothingBehind = "[^a-zA-Z\- #\n]"
    removeUncommonStops = " im | got | just | ive | th | hes | shes | its | dont | do |   *"

    text = text.lower()
    text = re.sub(removalRegex , '', text)
    text = re.sub(leaveNothingBehind , '', text)
    text = re.sub(removeUncommonStops , '', text)

    cleanText.append([i[0], text])
  
  # tf-idf
  cleanTextDF = pd.DataFrame(cleanText, columns=["itemID", "text"])
  vectorizer = CountVectorizer(stop_words='english') 
  counts = vectorizer.fit_transform(cleanTextDF.loc[:, "text"]) 
  countsDf = pd.DataFrame(counts.A, columns=vectorizer.get_feature_names_out())

  # drop uncommon words, get most important ones sorted
  allWords = list(countsDf.columns)
  recurringImportantWords = []
  
  # remove infrequent words
  for i, item in enumerate(countsDf.sum()):
    if(item >= 10):
      recurringImportantWords.append([allWords[i], item])

  recurringImportantWords = sorted(recurringImportantWords, key=lambda x: -x[1])
            # print(len(recurringImportantWords), recurringImportantWords)
            # print(np.array(recurringImportantWords[1]))

  # create TF-IDF vectors for each tweet
  tfidfVector = np.zeros((len(data.values), len(recurringImportantWords)), dtype=np.float32)
  indexList = []
  print(tfidfVector.shape)

  N = len(cleanTextDF.values)
  idf = np.log(N * np.reciprocal(np.array(np.array(recurringImportantWords)[:,1], dtype=np.float32)))

  onlyWords = np.array(recurringImportantWords)[:,0]
  print("Important words: ", onlyWords)


  for i, value in enumerate(cleanTextDF.values):
    # print(i,end="\r")
    words = value[1]
    indexList.append(value[0])

    # get per sentence word counts
    for word in words.split(" "):
      present = np.where(onlyWords == word)

      if len(present[0]) == 0:
        continue
      else:
        tfidfVector[i][int(present[0])] += 1


    print("Sentence: ", i, end="\r")
    tfidfVector[i] = np.log(1 + tfidfVector[i])
    tfidfVector[i] = tfidfVector[i] * idf

  # dimensionality reduction

  # normalise tfids first
  # normalise
  for i in range(1, tfidfVector.shape[1]):
    avg = np.mean(tfidfVector[:,i])
    tfidfVector[:,i] -= avg

    bigness = np.max(tfidfVector[:,i]) - np.min(tfidfVector[:,i])

    if bigness != 0:
      tfidfVector[:,i] /= bigness

  print("Performing SVD...")
  u, s, v = np.linalg.svd(tfidfVector)
  numDimensions = 25

  print("SVD complete")

  reducedTfidVector = np.empty((u.shape[0], numDimensions + 1))
  reducedTfidVector[:,0] = indexList
  reducedTfidVector[:,1:] = u[:,:numDimensions]

  return reducedTfidVector
   


# relevant metadata columns
# relCols = ["id", "fcount", "frndcount", "lcount", "user favcount", "verified", "scount", "rt count", "fav count", "sens"]
relCols = ["id", "fcount",  "frncount", "rt count", "fav count", ]

# return data and test splits
def getFeatures(data, percent=0.9):
  """
  Funtion to return a matrix of dimensions (number of tweets, number of chosen features)
  Input parameters to this funcion are to be chosen as per requirement (Example: Loaded dataframe of the dataset) 
  """

  """
  relevant columns:
  follower count
  friend count
  listed_count
  favourite count
  verified
  statuses_count
  rt count
  fac count
  sensitive
  """

  vecWidth = len(relCols)
  vecHeight = int(len(data.values) * percent)   # for data

  dataFeatureVector = np.zeros((vecHeight, vecWidth), dtype=np.int32)
  testFeatureVector = np.zeros((len(data.values) - vecHeight, vecWidth), dtype=np.int32)

  columns = list(data.columns)


  testCount = 0
  for i, item in enumerate(data.values):    
    feature = np.zeros(vecWidth)

    feature[0] = item[columns.index("itemID")]                                                    # ID
    feature[1] = item[columns.index("user")]["followers_count"]
    feature[2] = item[columns.index("user")]["friends_count"]
    feature[3] = item[columns.index("retweet_count")]
    feature[4] = item[columns.index("favorite_count")]
    # feature[5] = item[columns.index("user")]["friends_count"]
    # feature[6] = item[columns.index("user")]["verified"]
    # feature[7] = item[columns.index("user")]["statuses_count"]
    # feature[8] = item[columns.index("retweet_count")]
    # feature[9] = item[columns.index("favorite_count")]
    
    if i < vecHeight:
      dataFeatureVector[i] = feature
    else:
      testFeatureVector[testCount] = feature
      testCount += 1

  return dataFeatureVector, testFeatureVector

data = data.sample(frac=1)

reducedTfidfVector= tweet_vectoriser(data)




(11099, 1710)
Important words:  ['people' 'new' 'like' ... 'winners' 'writer' 'zimbabwe']
Performing SVD...
SVD complete


Perform KNN using the vector obtained from tweet_vectoriser() function. Following are the steps to be followed:

1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values.

In [119]:
# normalise
for i in range(1, reducedTfidfVector.shape[1]):
  avg = np.mean(reducedTfidfVector[:,i])
  reducedTfidfVector[:,i] -= avg

  bigness = np.max(reducedTfidfVector[:,i]) - np.min(reducedTfidfVector[:,i])
  reducedTfidfVector[:,i] /= bigness

print("Reduced tfid vecotr: ", reducedTfidfVector)

pd.options.display.max_colwidth = 1000

Reduced tfid vecotr:  [[ 7.03000000e+03  1.84513647e-02  7.35783956e-03 ...  3.18279093e-03
   2.83203222e-03 -2.67768532e-02]
 [ 9.75000000e+02  1.82159097e-02  3.36303764e-03 ... -1.14291537e-02
   4.16967891e-03 -3.11371604e-02]
 [ 7.30500000e+03  1.85309067e-02  6.72749855e-03 ... -4.43744854e-03
  -9.21414112e-03 -1.37778831e-02]
 ...
 [ 5.63200000e+03  1.86908755e-02  6.21310133e-03 ...  3.47224429e-03
  -6.95728744e-03 -7.35202988e-03]
 [ 4.08600000e+03  1.77339721e-02  8.84362533e-03 ... -8.06314066e-04
  -1.06263514e-03 -1.78023093e-02]
 [ 1.95000000e+02  1.88828151e-02  9.55180609e-03 ... -3.21500066e-03
  -5.07179616e-02  1.02345438e-01]]


In [120]:
# performing a k-nn search
# data was shuffled initially, no need to shuffle again

dataTfidf = reducedTfidfVector[ :int(0.90 * reducedTfidfVector.shape[0])  , :]
testTfidf = reducedTfidfVector[  int(0.90 * reducedTfidfVector.shape[0]): , :]

def textKNN(k, dataTfidf, testTfidf):
    correctPredictions = 0
    correctPositive = 0
    totalPredictions = len(nTestFeatureVector)
    totalPositives = 0
    predictedPositives = 0
    predictedNegatives = 0

    pptp = 0
    pntp = 0
    pptn = 0
    pntn = 0

    # get the features with the k smallest costs
    for testEntry in tqdm(testTfidf):
        allCosts = dataTfidf - testEntry
        allCosts = np.square(allCosts)
        allCosts = np.sum(allCosts[:, 1:], axis=1)          # 1st column is the ID

        sortedCosts = np.zeros((allCosts.shape[0], 2))
        sortedCosts[:,0] = np.sqrt(allCosts)
        sortedCosts[:,1] = dataTfidf[:,0]

        ind = np.argsort(sortedCosts[:,0])
        sortedCosts = sortedCosts[ind]

        closest = sortedCosts[:k,:]
        # check nbd classes
        tot = 0.0
        predictedClass = 0

        bleep = 0
        for i in closest:
            # print(i.shape)
            # print(i)
            # print(gt[int(i[1])])
            if gt[int(i[1])] == 1.0:
                bleep += 1
            tot += gt[int(i[1])]
        
        # if bleep > 0:
            # print(bleep, testEntry[0])

        if float(tot)/k < 0.5:
            predictedClass = 0
        else:
            predictedClass = 1

        if gt[int(testEntry[0])] == predictedClass:
            correctPredictions += 1
            if predictedClass == 1:
                correctPositive += 1

        if gt[int(testEntry[0])] == 1:
            totalPositives += 1

        if predictedClass == 1:
            predictedPositives += 1
            if gt[int(testEntry[0])] == 0:
                pptn += 1
            else:
                pptp += 1

        else:
            predictedNegatives += 1
            if gt[int(testEntry[0])] == 0:
                pntn += 1
            else:
                pntp += 1
    
    

    print("pptp: ", pptp, " | pptn: ", pptn, " | pntp: ", pntp, " | pntn: ", pntn)
    accuracy = float(pptp + pntn)/(pptp + pptn + pntp + pntn)
    positiveAccuracy = float(pptp)/(pptp+pntp)
    return accuracy, positiveAccuracy, correctPositive, totalPositives, predictedPositives, predictedNegatives

# loop over all ks, calculate accuracy and percent of correctly predicted positives
maxK = 10
for k in range(1, 10):
    accuracy, positiveAccuracy, correctPositive, tp, pp, pn = textKNN(k, dataTfidf, testTfidf)
    print("K = ", k, " | Accuracy = ", accuracy, " | Correctly predicted positives % = ", positiveAccuracy, " | correct positive number = ", correctPositive, " out of ", tp, " | pp = ", pp, " | pn = ", pn)


  0%|          | 0/1110 [00:00<?, ?it/s]/tmp/ipykernel_10439/2512531823.py:27: RuntimeWarning: divide by zero encountered in reciprocal
  sortedCosts[:,0] = -np.reciprocal(np.sqrt(allCosts))
100%|██████████| 1110/1110 [00:01<00:00, 874.02it/s]


pptp:  10  | pptn:  88  | pntp:  76  | pntn:  936
K =  1  | Accuracy =  0.8522522522522522  | Correctly predicted positives % =  0.11627906976744186  | correct positive number =  10  out of  86  | pp =  98  | pn =  1012


100%|██████████| 1110/1110 [00:01<00:00, 873.38it/s]


pptp:  12  | pptn:  164  | pntp:  74  | pntn:  860
K =  2  | Accuracy =  0.7855855855855856  | Correctly predicted positives % =  0.13953488372093023  | correct positive number =  12  out of  86  | pp =  176  | pn =  934


100%|██████████| 1110/1110 [00:01<00:00, 871.98it/s]


...

pptp:  0  | pptn:  1  | pntp:  86  | pntn:  1023
K =  7  | Accuracy =  0.9216216216216216  | Correctly predicted positives % =  0.0  | correct positive number =  0  out of  86  | pp =  1  | pn =  1109


100%|██████████| 1110/1110 [00:01<00:00, 864.42it/s]


pptp:  0  | pptn:  3  | pntp:  86  | pntn:  1021
K =  8  | Accuracy =  0.9198198198198199  | Correctly predicted positives % =  0.0  | correct positive number =  0  out of  86  | pp =  3  | pn =  1107


100%|██████████| 1110/1110 [00:01<00:00, 902.38it/s]

pptp:  0  | pptn:  0  | pntp:  86  | pntn:  1024
K =  9  | Accuracy =  0.9225225225225225  | Correctly predicted positives % =  0.0  | correct positive number =  0  out of  86  | pp =  0  | pn =  1110


## Part-3
### Subpart-1

Combine both the vectors obtained from the tweet_vectoriser() and get_features()

In [134]:
# your code here

def combineVectors(dataVector, tfidfVector):
    combinedVector = np.empty((dataVector.shape[0], dataVector.shape[1] + reducedTfidfVector.shape[1] - 1))
    for dataEntry, tfidfEntry in zip(dataVector, reducedTfidfVector):
        combinedVector[dataEntry[0],:len(dataEntry)] = dataEntry
        combinedVector[int(tfidfEntry[0]), len(dataEntry):] = tfidfEntry[:-1]
    
    return combinedVector

combinedVector = combineVectors(fullVector, reducedTfidfVector)

print(fullVector.shape, reducedTfidfVector.shape, combinedVector.shape)

(11099, 4) (11099, 26) (11099, 29)


Perform KNN using the vector obtained in the previous step. Following are the steps to be followed:

1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values.

In [136]:
# your code here

# performing a k-nn search
# data was shuffled initially, no need to shuffle again

dataCombinedVector = combinedVector[ :int(0.90 * combinedVector.shape[0])  , :]
testCombinedVector = combinedVector[  int(0.90 * combinedVector.shape[0]): , :]

def combinedKNN(k, dataTfidf, testTfidf):
    correctPredictions = 0
    correctPositive = 0
    totalPositives = 0
    predictedPositives = 0
    predictedNegatives = 0

    pptp = 0
    pntp = 0
    pptn = 0
    pntn = 0

    # get the features with the k smallest costs
    for testEntry in tqdm(testTfidf):
        allCosts = dataTfidf - testEntry
        allCosts = np.square(allCosts)
        allCosts = np.sum(allCosts[:, 1:], axis=1)          # 1st column is the ID

        sortedCosts = np.zeros((allCosts.shape[0], 2))
        sortedCosts[:,0] = -np.reciprocal(np.sqrt(allCosts))
        sortedCosts[:,1] = dataTfidf[:,0]

        ind = np.argsort(sortedCosts[:,0])
        sortedCosts = sortedCosts[ind]

        closest = sortedCosts[:k,:]
        # check nbd classes
        tot = 0.0
        predictedClass = 0

        bleep = 0
        for i in closest:
            # print(i.shape)
            # print(i)
            # print(gt[int(i[1])])
            if gt[int(i[1])] == 1.0:
                bleep += 1
            tot += gt[int(i[1])]
        
        # if bleep > 0:
            # print(bleep, testEntry[0])

        if float(tot)/k < 0.5:
            predictedClass = 0
        else:
            predictedClass = 1

        if gt[int(testEntry[0])] == predictedClass:
            correctPredictions += 1
            if predictedClass == 1:
                correctPositive += 1

        if gt[int(testEntry[0])] == 1:
            totalPositives += 1

        if predictedClass == 1:
            predictedPositives += 1
            if gt[int(testEntry[0])] == 0:
                pptn += 1
            else:
                pptp += 1

        else:
            predictedNegatives += 1
            if gt[int(testEntry[0])] == 0:
                pntn += 1
            else:
                pntp += 1
    
    

    print("pptp: ", pptp, " | pptn: ", pptn, " | pntp: ", pntp, " | pntn: ", pntn)
    accuracy = float(pptp + pntn)/(pptp + pptn + pntp + pntn)
    positiveAccuracy = float(pptp)/(pptp+pntp)
    return accuracy, positiveAccuracy, correctPositive, totalPositives, predictedPositives, predictedNegatives

# loop over all ks, calculate accuracy and percent of correctly predicted positives
maxK = 10
for k in range(1, 10):
    accuracy, positiveAccuracy, correctPositive, tp, pp, pn = textKNN(k, dataTfidf, testTfidf)
    print("K = ", k, " | Accuracy = ", accuracy, " | Correctly predicted positives % = ", positiveAccuracy, " | correct positive number = ", correctPositive, " out of ", tp, " | pp = ", pp, " | pn = ", pn)


  0%|          | 0/1110 [00:00<?, ?it/s]/tmp/ipykernel_10439/2512531823.py:27: RuntimeWarning: divide by zero encountered in reciprocal
  sortedCosts[:,0] = -np.reciprocal(np.sqrt(allCosts))
100%|██████████| 1110/1110 [00:01<00:00, 669.43it/s]


pptp:  10  | pptn:  88  | pntp:  76  | pntn:  936
K =  1  | Accuracy =  0.8522522522522522  | Correctly predicted positives % =  0.11627906976744186  | correct positive number =  10  out of  86  | pp =  98  | pn =  1012


100%|██████████| 1110/1110 [00:01<00:00, 695.62it/s]


pptp:  12  | pptn:  164  | pntp:  74  | pntn:  860
K =  2  | Accuracy =  0.7855855855855856  | Correctly predicted positives % =  0.13953488372093023  | correct positive number =  12  out of  86  | pp =  176  | pn =  934


100%|██████████| 1110/1110 [00:01<00:00, 709.91it/s]


...

pptp:  0  | pptn:  1  | pntp:  86  | pntn:  1023
K =  7  | Accuracy =  0.9216216216216216  | Correctly predicted positives % =  0.0  | correct positive number =  0  out of  86  | pp =  1  | pn =  1109


100%|██████████| 1110/1110 [00:01<00:00, 714.34it/s]


pptp:  0  | pptn:  3  | pntp:  86  | pntn:  1021
K =  8  | Accuracy =  0.9198198198198199  | Correctly predicted positives % =  0.0  | correct positive number =  0  out of  86  | pp =  3  | pn =  1107


100%|██████████| 1110/1110 [00:01<00:00, 749.68it/s]

pptp:  0  | pptn:  0  | pntp:  86  | pntn:  1024
K =  9  | Accuracy =  0.9225225225225225  | Correctly predicted positives % =  0.0  | correct positive number =  0  out of  86  | pp =  0  | pn =  1110


### Subpart-2

Explain the differences between the accuracies obtained in each part above based on the features used.